In [1]:
import pandas as pd

In [3]:
history

,precip_fengqiao,precip_suzhou,T2M,T2M_MAX,T2M_MIN,ETo_HS,runoff_fengqiao,runoff_suzhou
time,,,,,,,,
1989-01-01,0.0,0.0,4.81,10.07,1.12,1.22,0.018179,0.018179
1989-01-02,0.0,0.0,5.83,12.39,0.13,1.49,0.036330,0.036330
1989-01-03,0.0,0.0,6.70,12.26,3.40,1.32,0.054453,0.054453
1989-01-04,0.0,0.0,7.05,12.15,4.04,1.29,0.072550,0.072550
1989-01-05,0.9,0.5,7.32,12.61,3.19,1.41,0.090644,0.090633
...,...,...,...,...,...,...,...,...
2022-12-27,0.0,0.0,2.28,8.31,-2.52,1.18,72.722993,67.458859
2022-12-28,0.0,0.0,3.28,7.74,0.19,1.03,72.725316,67.461033
2022-12-29,0.0,0.0,2.78,6.32,0.17,0.91,72.727635,67.463204


In [2]:
# load data
history = pd.read_csv('../data/results/history.csv', index_col=0)
ssp245 = pd.read_csv('../data/results/ssp245.csv', index_col=0)
ssp585 = pd.read_csv('../data/results/ssp585.csv', index_col=0)
history_wl = pd.read_csv('../data/results/history_wl.csv', index_col=0)

In [ ]:
from darts import TimeSeries
from darts.models import RNNModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, r2_score
from darts.metrics import rmse
import matplotlib.pyplot as plt

# 从数据框创建时间序列
series_history = TimeSeries.from_dataframe(history.reset_index(), 'time')
series_wl = TimeSeries.from_dataframe(history_wl.reset_index(), 'time', ['water_level'])

# 数据预处理
scaler_history = Scaler()
scaler_wl = Scaler()

series_history_scaled = scaler_history.fit_transform(series_history)
series_wl_scaled = scaler_wl.fit_transform(series_wl)

wl_train, _ = series_wl_scaled.split_before(0.8)

def evaluate_model(model,past_covariates=None, future_covariates=None):
    # We backtest the model on the last 20% of the flow series, with a horizon of 10 steps:
    backtest = model.historical_forecasts(series=series_wl_scaled, 
                                          past_covariates=past_covariates,
                                          future_covariates=future_covariates,
                                          start=0.8, 
                                          retrain=False,
                                          verbose=True, 
                                          forecast_horizon=10)
    
    # Iterate over the list of TimeSeries objects and call the plot method on each one
    for ts in [series_wl_scaled[-len(backtest)-100:], backtest]:
        ts.plot(label='backtest (n=10)')
    
    print('Backtest RMSE = {}'.format(rmse(series_wl_scaled, backtest)))    

In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.metrics import rmse
import torch
from darts.models import BlockRNNModel


# load data
history = pd.read_csv('../data/results/history.csv', index_col=0)
ssp245 = pd.read_csv('../data/results/ssp245.csv', index_col=0)
ssp585 = pd.read_csv('../data/results/ssp585.csv', index_col=0)
history_wl = pd.read_csv('../data/results/history_wl.csv', index_col=0)

# 从数据框创建时间序列
series_history = TimeSeries.from_dataframe(history.reset_index(), 'time')
series_wl = TimeSeries.from_dataframe(history_wl.reset_index(), 'time', ['water_level'])



# 数据预处理
scaler_history = Scaler()
scaler_wl = Scaler()

series_history_scaled = scaler_history.fit_transform(series_history)
series_wl_scaled = scaler_wl.fit_transform(series_wl)

# Determine the split point
split_idx = int(len(series_wl_scaled) * 0.8)
split_date = series_wl_scaled.time_index[split_idx]

# split data into training and validation sets
wl_train = series_wl_scaled.split_before(split_date)
wl_val = series_wl_scaled.split_after(split_date)

covariates_history_train = series_history_scaled.split_before(split_date)
covariates_history_val = series_history_scaled.split_after(split_date)



torch.set_float32_matmul_precision('medium')
model = BlockRNNModel(
    model='LSTM', 
    input_chunk_length=30, 
    output_chunk_length=10,
    n_rnn_layers=2,
    random_state=42,
    pl_trainer_kwargs = {"accelerator": "gpu", "devices": [0]}  
)

model.fit(
    series=wl_train,
    past_covariates=covariates_history_train,
    verbose=True,
    epochs=5
)


# 使用模型进行预测
prediction = model.predict(n=len(wl_val), series=wl_val, past_covariates=covariates_history_val)



# 计算 RMSE
error = rmse(prediction, wl_val)

print(f'RMSE: {error}')




In [ ]:
history

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import statsmodels.api as sm
import numpy as np

features_train, features_val, target_train, target_val = train_test_split(
    history,
    history_wl,
    test_size=0.2,
    random_state=42
)
# 添加常数列作为截距项
features = sm.add_constant(features_train)
# 拟合VAR模型
model = sm.OLS(target_train, features)
results = model.fit()

features_val = sm.add_constant(features_val)

predictions = results.predict(features_val)

rmse =  np.sqrt(mean_squared_error(target_val, predictions))

# R2 score
r2 = r2_score(target_val, predictions)

print(f'RMSE: {rmse}')
print(f'R2: {r2}')

In [ ]:
wl_ssp245.max()

In [ ]:
suzhou = pd.read_csv('../data/intermediate/suzhou_interpolated.csv', index_col=0)

In [ ]:
suzhou['water_level'].plot()

In [ ]:
history_wl.index

In [ ]:
history

In [ ]:
from prophet import Prophet
df = pd.DataFrame({'ds': history_wl.index, 'y': history_wl['water_level']})
df['runoff_fengqiao'] = history['runoff_fengqiao']
df['runoff_suzhou'] = history['runoff_suzhou']
df['precip_fengqiao'] = history['precip_fengqiao']
df['precip_suzhou'] = history['precip_suzhou']
df['T2M'] = history['T2M']
df['T2M_MAX'] = history['T2M_MAX']
df['T2M_MIN'] = history['T2M_MIN']
df['ETo_HS'] = history['ETo_HS']
from sklearn.model_selection import train_test_split

# 设置随机种子，以确保结果可重现
random_seed = 42

# 使用80%的数据作为训练集，20%的数据作为测试集
train_size = 0.8

# 注意，你需要先将'ds'列转换为日期类型
df['ds'] = pd.to_datetime(df['ds'])

# 按时间顺序排序，确保拆分是按照时间序列的顺序进行的
df = df.sort_values('ds')

# 计算训练集的大小
train_len = int(train_size * len(df))

# 划分数据集
df_train = df.iloc[:train_len]
df_test = df.iloc[train_len:]

# 初始化Prophet模型
m = Prophet()
m.add_regressor('runoff_fengqiao')
m.add_regressor('runoff_suzhou')
m.add_regressor('precip_fengqiao')
m.add_regressor('precip_suzhou')
m.add_regressor('T2M')
m.add_regressor('T2M_MAX')
m.add_regressor('T2M_MIN')
m.add_regressor('ETo_HS')


m.fit(df_train)

In [ ]:
df

In [ ]:
# 创建一个未来的时间段，长度为测试集的长度
future = m.make_future_dataframe(periods=len(df_test))
future['runoff_fengqiao'] = df['runoff_suzhou'].values
future['runoff_suzhou'] = df['runoff_suzhou'].values
future['precip_fengqiao'] = df['precip_fengqiao'].values
future['precip_suzhou'] = df['precip_suzhou'].values
future['T2M'] = df['T2M'].values
future['T2M_MAX'] = df['T2M_MAX'].values
future['T2M_MIN'] = df['T2M_MIN'].values
future['ETo_HS'] = df['ETo_HS'].values
forecast = m.predict(future)

forecast_test = forecast.loc[forecast['ds'].isin(df_test['ds'])]

from sklearn.metrics import mean_squared_error
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric
rmse = mean_squared_error(df_test['y'], forecast_test['yhat'], squared=False)
print('RMSE: ', rmse)

r2 = r2_score(df_test['y'], forecast_test['yhat'])


In [ ]:
r2 = r2_score(df_test['y'], forecast_test['yhat'])

In [ ]:
# 你也可以使用Prophet的内置交叉验证
df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')
df_p = performance_metrics(df_cv)
print(df_p)

In [ ]:
ssp585

In [ ]:
future_ssp585 = pd.DataFrame({'ds': ssp585.index,'runoff_fengqiao':ssp585['runoff_fengqiao'],'runoff_suzhou':ssp585['runoff_suzhou'],'precip_fengqiao':ssp585['precip_fengqiao'],'precip_suzhou':ssp585['precip_suzhou'],'T2M':ssp585['T2M'],'T2M_MAX':ssp585['T2M_MAX'],'T2M_MIN':ssp585['T2M_MIN'],'ETo_HS':ssp585['ETo_HS']})


forecast_ssp585 = m.predict(future_ssp585)

forecast_ssp585['yhat'].plot()

In [ ]:
import itertools
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative'],
    'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0],
}


# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(df_train)  # Fit model with given params
    df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

In [ ]:
best_params = tuning_results.loc[tuning_results['rmse'].idxmin()]

best_params = tuning_results.loc[tuning_results['rmse'].idxmin()]
best_rmse = best_params['rmse']
print("Best Parameters:")
print(best_params)
print("Best RMSE:", best_rmse)


In [ ]:
best_params = tuning_results.loc[tuning_results['rmse'].idxmin()]

prophet_params = {
    'changepoint_prior_scale': best_params['changepoint_prior_scale'],
    'seasonality_prior_scale': best_params['seasonality_prior_scale'],
    'seasonality_mode': best_params['seasonality_mode'],
    'holidays_prior_scale': best_params['holidays_prior_scale']
}

m = Prophet(**prophet_params).fit(df_train)  # 使用最佳参数拟合模型

future_ssp585 = pd.DataFrame({'ds': ssp585.index, 'runoff_fengqiao': ssp585['runoff_fengqiao'], 'runoff_suzhou': ssp585['runoff_suzhou'], 'precip_fengqiao': ssp585['precip_fengqiao'], 'precip_suzhou': ssp585['precip_suzhou'], 'T2M': ssp585['T2M'], 'T2M_MAX': ssp585['T2M_MAX'], 'T2M_MIN': ssp585['T2M_MIN'], 'ETo_HS': ssp585['ETo_HS']})

forecast_ssp585 = m.predict(future_ssp585)

forecast_ssp585['yhat'].plot()

In [ ]:
history_wl.plot()

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# 提取特征和目标值
X = series_history.values().reshape(-1, 1)
y = series_wl.values().reshape(-1, 1)

# 数据归一化
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# 训练SVR模型
svr = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr.fit(X, y.ravel())

# 用模型进行预测
y_pred = svr.predict(X)

# 将预测结果从归一化状态恢复到原始状态
y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(y_pred)